# Dataflow

## Setup IAM and networking for Dataflow jobs

### Create a Cloud Storage bucket

In [ ]:
gcloud auth list
gcloud config list project


PROJECT=`gcloud config list --format 'value(core.project)'`
USER_EMAIL=`gcloud config list account --format 'value(core.account)'`
REGION=us-central1

gsutil mb -p $PROJECT -b on gs://$PROJECT

### Create a virtual environment

In [ ]:
sudo apt-get install -y python3-venv
python3 -m venv df-env
source df-env/bin/activate

python3 -m pip install -q --upgrade pip setuptools wheel
python3 -m pip install apache-beam[gcp]

### Launch a Dataflow job

In [ ]:
gcloud projects get-iam-policy $PROJECT \
--format='table(bindings.role)' \
--flatten='bindings[].members' \
--filter='bindings.members:$USER_EMAIL'


gcloud projects add-iam-policy-binding $PROJECT \
--member=user:$USER_EMAIL \
--role=roles/dataflow.admin


python3 -m apache_beam.examples.wordcount \
--input=gs://dataflow-samples/shakespeare/kinglear.txt \
--output=gs://$PROJECT/results/outputs \
--runner=DataflowRunner \
--project=$PROJECT \
--temp_location=gs://$PROJECT/tmp/ \
--region=$REGION

### Launch in private IPs

In [ ]:
gcloud projects add-iam-policy-binding $PROJECT \
--member=user:$USER_EMAIL \
--role=roles/compute.networkAdmin


gcloud compute networks subnets update default \
--region=$REGION \
--enable-private-ip-google-access


python3 -m apache_beam.examples.wordcount \
--input=gs://dataflow-samples/shakespeare/kinglear.txt \
--output=gs://$PROJECT/results/outputs \
--runner=DataflowRunner \
--project=$PROJECT \
--temp_location=gs://$PROJECT/tmp/ \
--region=$REGION \
--no_use_public_ips \
--network default

## Extract-Transform-Load pipeline

### Setting up virtual environment and dependencies 

In [ ]:
git clone https://github.com/GoogleCloudPlatform/training-data-analyst
cd ~/training-data-analyst/quests/dataflow_python/

# Change directory into the lab
cd 1_Basic_ETL/lab
export BASE_DIR=$(pwd)

sudo apt-get install -y python3-venv
python3 -m venv df-env
source df-env/bin/activate

python3 -m pip install -q --upgrade pip setuptools wheel
python3 -m pip install apache-beam[gcp]

# Dataflow API is enabled.
gcloud services enable dataflow.googleapis.com

### First pipeline

In [ ]:
# Change to the directory containing the relevant code
cd $BASE_DIR/../..
# Create GCS buckets and BQ dataset
source create_batch_sinks.sh
# Run a script to generate a batch of web server log events
bash generate_batch_events.sh
# Examine some sample events
head events.json

In [ ]:
%%writefile 1_Basic_ETL/lab/my_pipeline.py
import argparse
import time
import logging
import json
import apache_beam as beam
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.runners import DataflowRunner, DirectRunner

# ### main

def run():
    # Command line arguments
    parser = argparse.ArgumentParser(description='Load from Json into BigQuery')
    parser.add_argument('--project',required=True, help='Specify Google Cloud project')
    parser.add_argument('--region', required=True, help='Specify Google Cloud region')
    parser.add_argument('--stagingLocation', required=True, help='Specify Cloud Storage bucket for staging')
    parser.add_argument('--tempLocation', required=True, help='Specify Cloud Storage bucket for temp')
    parser.add_argument('--runner', required=True, help='Specify Apache Beam Runner')

    opts = parser.parse_args()

    # Setting up the Beam pipeline options
    options = PipelineOptions()
    options.view_as(GoogleCloudOptions).project = opts.project
    options.view_as(GoogleCloudOptions).region = opts.region
    options.view_as(GoogleCloudOptions).staging_location = opts.stagingLocation
    options.view_as(GoogleCloudOptions).temp_location = opts.tempLocation
    options.view_as(GoogleCloudOptions).job_name = '{0}{1}'.format('my-pipeline-',time.time_ns())
    options.view_as(StandardOptions).runner = opts.runner

    # Static input and output
    input = 'gs://{0}/events.json'.format(opts.project)
    output = '{0}:logs.logs'.format(opts.project)

    # Table schema for BigQuery
    table_schema = {
        "fields": [
            {
                "name": "ip",
                "type": "STRING"
            },
            {
                "name": "user_id",
                "type": "STRING"
            },
            {
                "name": "lat",
                "type": "FLOAT"
            },
            {
                "name": "lng",
                "type": "FLOAT"
            },
            {
                "name": "timestamp",
                "type": "STRING"
            },
            {
                "name": "http_request",
                "type": "STRING"
            },
            {
                "name": "http_response",
                "type": "INTEGER"
            },
            {
                "name": "num_bytes",
                "type": "INTEGER"
            },
            {
                "name": "user_agent",
                "type": "STRING"
            }
        ]
    }

    # Create the pipeline
    p = beam.Pipeline(options=options)

    '''
    Steps:
    1) Read something
    2) Transform something
    3) Write something
    '''

    (p
        | 'ReadFromGCS' >> beam.io.ReadFromText(input)
        | 'ParseJson' >> beam.Map(lambda line: json.loads(line))
        | 'WriteToBQ' >> beam.io.WriteToBigQuery(
            output,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE
            )
    )

    logging.getLogger().setLevel(logging.INFO)
    logging.info("Building pipeline ...")

    p.run()

if __name__ == '__main__':
    run()

In [ ]:
cd $BASE_DIR
# Set up environment variables
export PROJECT_ID=$(gcloud config get-value project)
# Run the pipeline
python3 my_pipeline.py \
  --project=${PROJECT_ID} \
  --region=us-central1 \
  --stagingLocation=gs://$PROJECT_ID/staging/ \
  --tempLocation=gs://$PROJECT_ID/temp/ \
  --runner=DataflowRunner

## Beam Notebooks

Include the `interactive_runner` and `interactive_beam` modules in notebook.

In [ ]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [ ]:
# Set the recording duration to 10 min
ib.options.recording_duration = '10m'

# Set the recording size limit to 1 GB
ib.options.recording_size_limit = 1e9

In [ ]:
words = p | "read" >> beam.io.ReadFromPubSub(topic=topic)

windowed_words = (words | "window" >> beam.WindowInto(beam.window.FixedWindows(10)))

windowed_words_counts = (windowed_words | "count" >> beam.combiners.Count.PerElement())

In [ ]:
# Materializes the resulting PCollection in a table
ib.show(windowed_word_counts, include_window_info=True)

# Load the output in a Pandas DataFrame
ib.collect(windowed_word_counts, include_window_info=True)

# Visualize the data in the Notebook
ib.show(windowed_word_counts, include_window_info=True, visualize_data=True)

In [ ]:
# Import the production Dataflow runner
from apache_beam.runners import DataflowRunner

# Set up Apache Beam pipeline options
options = pipeline_options.PipelineOptions()

# Run the pipeline
runner = DataflowRunner()
runner.run_pipeline(p, options=options)

In [ ]:
storeSales = p | beam.io.ReadFromText("purchases-store")
               | beam.Map(lambda s: ...)

onlineSales = p | beam.io.ReadFromText("purchase-online")
                | beam.Map(lambda s: ...)
    
topSales = (storeSales, onlineSales)
                | beam.Flatten()
                | beam.Combiners.Count.perKey()
                | beam.Combiners.Top.of(10, key=lambda x: x[1])
            
topSales        | beam.io.WriteToBigQuery(topSales)